In [11]:
using NCDatasets
using PyPlot
using PyCall
const plt = PyPlot
ccrs = pyimport("cartopy.crs")
cfeature = pyimport("cartopy.feature")
coast = cfeature.GSHHSFeature(scale="i")
mpl = pyimport("matplotlib")
mpl.style.use("./fairease.mplstyle")
cmocean = pyimport("cmocean")

PyObject <module 'cmocean' from '/home/ctroupin/.julia/conda/3/x86_64/lib/python3.12/site-packages/cmocean/__init__.py'>

In [12]:
using Conda
Conda.add("cmocean")

[ Info: Running `conda install -y cmocean` in root environment


Channels:
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.



In [13]:
theproj = ccrs.EuroPP()
datacrs = ccrs.PlateCarree()

PyObject <Projected CRS: +proj=eqc +ellps=WGS84 +a=6378137.0 +lon_0=0.0 +to ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (unknown)
- N[north]: Northing (unknown)
- h[up]: Ellipsoidal height (metre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Equidistant Cylindrical
Datum: Unknown based on WGS 84 ellipsoid
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich


/home/ctroupin/.julia/conda/3/x86_64/lib/python3.12/site-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/110m_physical/ne_110m_coastline.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)


In [15]:
nc = NCDataset("../data/NorthAdriatic_bathy.nc", "r")
lonb = nc["lon"][:]
latb = nc["lat"][:]
bathy = nc["bat"][:,:]
close(nc)

closed Dataset

In [20]:
bathy2plot1 = copy(bathy);
bathy2plot2 = copy(bathy);

bathy2plot1[bathy2plot1.>=0] .= 0;
bathy2plot2[bathy2plot1.<0] .= 0;

mpl = pyimport("matplotlib")
ls = mpl.colors.LightSource(azdeg=315, altdeg=45)
cmap = plt.cm.Greys
shadedtopo = ls.shade(bathy2plot2, cmap=cmap, blend_mode="soft", vert_exag=0.1)
shadedtopo = permutedims(shadedtopo, [2, 1, 3]);

In [22]:
fig = plt.figure()
ax = plt.subplot(111, projection=theproj)
pcm = ax.pcolormesh(lonb, latb, bathy2plot1', transform=datacrs, cmap=cmocean.cm.deep_r)
cb = plt.colorbar(pcm)
cb.set_label("Depth (m)", rotation=0, ha="left")
gl = ax.gridlines(draw_labels=true, linewidth=.5, color=".25", linestyle="-.", zorder=2)
ax.imshow(shadedtopo, extent=[minimum(lonb), maximum(lonb), minimum(latb), maximum(latb)], 
transform=datacrs, zorder=5)
gl.top_labels = false
gl.right_labels = false
#ax.add_feature(coast, linewidth=0.25, facecolor=".65", zorder=3)
ax.set_title("EMODnet Bathymetry")

LoadError: PyError ($(Expr(:escape, :(ccall(#= /home/ctroupin/.julia/packages/PyCall/1gn3u/src/pyfncall.jl:43 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'ImportError'>
ImportError('Using image transforms requires either pykdtree or scipy.')
  File "/home/ctroupin/.julia/conda/3/x86_64/lib/python3.12/site-packages/cartopy/mpl/geoaxes.py", line 307, in wrapper
    return func(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ctroupin/.julia/conda/3/x86_64/lib/python3.12/site-packages/cartopy/mpl/geoaxes.py", line 1312, in imshow
    from cartopy.img_transform import warp_array
  File "/home/ctroupin/.julia/conda/3/x86_64/lib/python3.12/site-packages/cartopy/img_transform.py", line 20, in <module>
    raise ImportError("Using image transforms requires either "


In [23]:
extrema(bathy)

(-264, 2135)